# 问题

In [1]:
link = "https://gkcx.eol.cn/soudaxue/queryProvinceScoreNum.html?schoolflag=&page=1&schoolyear=&argyear="

这个链接返回的页面是动态页面，其 HTML 文件本身只有空的表格和 JavaScript 代码，没有数据。当浏览器渲染页面时会执行 JavaScript 代码，此时 JavaScript 代码从网站服务器获取数据，然后展现在页面上。Python 并不会执行 JavaScript，所以 pandas 的 read_html 函数看到的是没有数据的页面。

In [2]:
import pandas as pd
pd.read_html(link)

[Empty DataFrame
 Columns: [学校名称, 招生地区, 文理科, 年份, 录取批次, 录取人数, 录取平均分, 高校对比]
 Index: []]

# 抓取动态页面

抓取动态页面的数据就是要找到网站的API,一般来说这些API都是返回JSON文件。比如说，我在中国教育在线网站查询北京大学的时候，有一个链接返回的是JSON 资源。

![image.png](query.png)

这个链接返回的JSON文件就是我们想要的数据：

![json.png](json.png)

分析这个API的参数

```
https://data-gkcx.eol.cn/soudaxue/queryProvinceScore.html?messtype=jsonp&lunum=1&callback=jQuery18306243269813396837_1527828613487&provinceforschool=&schooltype=&page=1&size=10&keyWord=%E5%8C%97%E4%BA%AC%E5%A4%A7%E5%AD%A6&schoolproperty=&schoolflag=&province=%E5%8C%97%E4%BA%AC&fstype=%E7%90%86%E7%A7%91&zhaoshengpici=&fsyear=&_=1527828616015
```

发现参数 messtype 的值是 jsonp，意思就是“请求的消息类型是 jsonp”，jsonp 就是 json 但是多了些我们不需要的东西，Python 不好直接处理，我们可以把 messtype 改成 json, 这个API就能直接返回 json 了。参数 page 和 size 关于分页的，网站默认是 `size=10` `page=1`，大概就是每页10行，请求第一页的数据。

In [3]:
# pip install requests, or conda install requests
import requests

url = r"https://data-gkcx.eol.cn/soudaxue/queryProvinceScore.html?messtype=json"
# 每页10行，请求第2页
r = requests.get(url, data = {"page": 2, "size": 10})
r.json()

{'totalRecord': {'num': '234143'},
 'school': [{'schoolid': '31',
   'schoolname': '北京大学',
   'localprovince': '云南',
   'province': '北京',
   'studenttype': '文科',
   'year': '2016',
   'batch': '本科提前批',
   'var': '671',
   'var_score': '671',
   'max': '--',
   'min': '--',
   'num': '0',
   'fencha': '0',
   'provincescore': '0',
   'url': 'http://gkcx.eol.cn/schoolhtm/schoolAreaPoint/31/10001/10034/10149.htm?779822'},
  {'schoolid': '31',
   'schoolname': '北京大学',
   'localprovince': '西藏',
   'province': '北京',
   'studenttype': '文科',
   'year': '2016',
   'batch': '本科提前批',
   'var': '629',
   'var_score': '629',
   'max': '--',
   'min': '--',
   'num': '0',
   'fencha': '0',
   'provincescore': '0',
   'url': 'http://gkcx.eol.cn/schoolhtm/schoolAreaPoint/31/10025/10034/10149.htm?780715'},
  {'schoolid': '31',
   'schoolname': '北京大学',
   'localprovince': '陕西',
   'province': '北京',
   'studenttype': '文科',
   'year': '2016',
   'batch': '本科提前批',
   'var': '656',
   'var_score': '656',
  

Json 数据很好操作

In [4]:
row = "{schoolname}, {localprovince}, {province}, {studenttype}, {year}, {batch}, {var}, {var_score}"

for s in r.json()["school"]:
    print(row.format(**s))

北京大学, 云南, 北京, 文科, 2016, 本科提前批, 671, 671
北京大学, 西藏, 北京, 文科, 2016, 本科提前批, 629, 629
北京大学, 陕西, 北京, 文科, 2016, 本科提前批, 656, 656
北京大学, 河北, 北京, 文科, 2016, 本科提前批, 665, 665
北京大学, 山西, 北京, 文科, 2016, 本科提前批, 610, 610
北京大学, 黑龙江, 北京, 文科, 2016, 本科提前批, 628, 628
北京大学, 河南, 北京, 文科, 2016, 本科提前批, 630, 630
北京大学, 河北, 北京, 理科, 2016, 本科提前批, 682, 682
北京大学, 湖北, 北京, 文科, 2016, 本科提前批, 640, 640
北京大学, 湖南, 北京, 文科, 2016, 本科提前批, 643, 643
